In [3]:
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os
import torch.nn.functional as F
import cv2
import numpy as np
import torch.utils.data as Data
from PIL import  Image,ImageDraw
import numpy as np
import random

Batch_Size = 1
num_epochs = 100
learning_rate = 1e-3
samples = 2000

data1 = []
data2= []


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% --  Generate dataset


for i in range(samples):
    
    switch = random.randint(0,1)
    if switch==0:
        img = Image.new('RGB', (28, 28), color = 'white')
        row,col,ch= np.shape(img)
        mean = random.randint(1,6)
        var = random.random()
        sigma = var**0.5
        gauss_pos1 = np.random.normal(mean,sigma,(row,col,ch))
        noisy_pos1 = img + gauss_pos1
        im2_pos1 = Image.fromarray(noisy_pos1,'RGB')
        draw = ImageDraw.Draw(im2_pos1)
        draw.rectangle([9,12,12,15],fill='red',outline='red')
        data1.append(np.asarray(im2_pos1))

        img_1 = Image.new('RGB', (28, 28), color = 'white')
        row_1,col_1,ch_1= np.shape(img_1)
        mean_1 = random.randint(1,6)
        var_1 = random.random()
        sigma_1 = var_1**0.5
        gauss_pos2 = np.random.normal(mean_1,sigma_1,(row_1,col_1,ch_1))
        noisy_pos2 = img_1 + gauss_pos2
        im2_pos2 = Image.fromarray(noisy_pos2,'RGB')
        draw_1 = ImageDraw.Draw(im2_pos2)
        draw_1.rectangle([9,12,12,15], fill='red', outline='red')
        data2.append(np.asarray(im2_pos2))
    elif switch==1:
        img_blank = Image.new('RGB', (28, 28), color = 'white')
        row_blank,col_blank,ch_blank= np.shape(img_blank)
        mean_blank = random.randint(1,6)
        var_blank = random.random()
        sigma_blank = var_blank**0.5
        gauss_blank_pos1 = np.random.normal(mean_blank,sigma_blank,(row_blank,col_blank,ch_blank))
        noisy_blank_pos1 = img_blank + gauss_blank_pos1
        im2_blank_pos1 = Image.fromarray(noisy_blank_pos1,'RGB')
        data1.append(np.asarray(im2_blank_pos1))

        img_blank_1 = Image.new('RGB', (28, 28), color = 'white')
        row_blank_1,col_blank_1,ch_blank_1= np.shape(img_blank_1)
        mean_blank_1 = random.randint(1,6)
        var_blank_1 = random.random()
        sigma_blank_1 = var_blank_1**0.5
        gauss_blank_pos2 = np.random.normal(mean_blank_1,sigma_blank_1,(row_blank_1,col_blank_1,ch_blank_1))
        noisy_blank_pos2 = img_blank_1 + gauss_blank_pos2
        im2_blank_pos2 = Image.fromarray(noisy_blank_pos2,'RGB')
        data2.append(np.asarray(im2_blank_pos2))


#%%%%%%%%%%%%%%%%%%%%%%%%%%% Noisy data
img = Image.new('RGB', (28, 28), color = 'white')
row,col,ch= np.shape(img)
mean = 0
var = 1
sigma = var**0.5
gauss = np.random.normal(mean,sigma,(row,col,ch))
noisy = img + gauss
im2 = Image.fromarray(noisy,'RGB')
b = np.asarray(im2)
b = np.array(b, dtype=np.float32)
b = np.transpose(b,(2,0,1))
b = torch.from_numpy(b)

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%       

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Data Preparation

y0 = np.array(data1, dtype=np.float32)
y1 = np.array(data2, dtype=np.float32)
y0 = np.transpose(y0, (0, 3, 1, 2))# This to make channels first for CNN
y1 = np.transpose(y1, (0, 3, 1, 2)) 


print(y0.shape)

x = torch.from_numpy(y0)
y = torch.from_numpy(y1)


torch_dataset = Data.TensorDataset(x,y)


loader = Data.DataLoader(

    dataset=torch_dataset,

    batch_size=Batch_Size,

    shuffle=True,

    num_workers=0,

)


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Defining NN

class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, 3, stride=1, padding=1),
            nn.MaxPool2d(3),  
            nn.Conv2d(16, 8, 3, stride=1, padding=1), 
            nn.MaxPool2d(3, stride=1),
            nn.Conv2d(8, 4, 3, stride=1, padding=1),
            nn.MaxPool2d(3, stride=1),
            nn.Conv2d(4, 2, 3, stride=1, padding=1),
            nn.MaxPool2d(3, stride=1)
        )

    def forward(self, x):
        x = self.encoder(x)
        return x

class auto_decoder(nn.Module):
    def __init__(self):
        super(auto_decoder,self).__init__()
        ## decoder layers ##
        ## a kernel of 2 and a stride of 2 will increase the spatial dims by 2
        self.t_conv1 = nn.ConvTranspose2d(4, 16, 3,padding=1)
        self.t_conv2 = nn.ConvTranspose2d(16, 3, 3,padding=1)
        
    def forward(self, x):
        ## decode ##
        # add transpose conv layers, with relu activation function
        x = F.relu(self.t_conv1(x))
        # output layer (with sigmoid for scaling from 0 to 1)
        x = F.sigmoid(self.t_conv2(x))
                
        return x
    

model = autoencoder()
model_1 = auto_decoder()

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Loss Function

def h_score(fx, gy):
 
    fx = fx - fx.mean(0)

    gy = gy - gy.mean(0)

    Nsamples = fx.size(0)

    covf = torch.matmul((torch.transpose(fx,3,3)), fx) / Nsamples

    covg = torch.matmul((torch.transpose(gy,3,3)), (gy)) / Nsamples

    h = -2 * torch.mean((fx * gy).sum(1)) + (covf * covg).sum()

    return h


optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=1e-3)

criterion = nn.MSELoss()
optimizer_1 = torch.optim.Adam(model_1.parameters(), lr=learning_rate,weight_decay=1e-5)


for epoch in range(num_epochs):
    for x,y in loader:
        optimizer.zero_grad()
        # ===================forward=====================
        #loss = criterion(output1, img)
        loss = h_score(model(x),model(y))
        # ===================backward====================

        #optimizer_1.zero_grad()
        loss.backward()
        optimizer.step()
        #optimizer_1.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'.format(epoch+1, num_epochs, loss.item()))
    print("x: ", model(x[0].reshape(1,3,28,28)))
    print("y:", model(y[0].reshape(1,3,28,28)))
    print("b:", model(b.reshape(1,3,28,28)))


(2000, 3, 28, 28)
epoch [1/100], loss:0.0000
x:  tensor([[[[-4.3417e-40, -4.3417e-40, -4.3417e-40],
          [-4.3417e-40, -4.3417e-40, -4.3417e-40],
          [-4.3417e-40, -4.3417e-40, -4.3417e-40]],

         [[-1.8324e-39, -1.8324e-39, -1.8324e-39],
          [-1.8324e-39, -1.8324e-39, -1.8324e-39],
          [-1.8324e-39, -1.8324e-39, -1.8324e-39]]]],
       grad_fn=<MaxPool2DWithIndicesBackward>)
y: tensor([[[[-4.3417e-40, -4.3417e-40, -4.3417e-40],
          [-4.3417e-40, -4.3417e-40, -4.3417e-40],
          [-4.3417e-40, -4.3417e-40, -4.3417e-40]],

         [[-1.8324e-39, -1.8324e-39, -1.8324e-39],
          [-1.8324e-39, -1.8324e-39, -1.8324e-39],
          [-1.8324e-39, -1.8324e-39, -1.8324e-39]]]],
       grad_fn=<MaxPool2DWithIndicesBackward>)
b: tensor([[[[-4.3417e-40, -4.3417e-40, -4.3417e-40],
          [-4.3417e-40, -4.3417e-40, -4.3417e-40],
          [-4.3417e-40, -4.3417e-40, -4.3417e-40]],

         [[-1.8324e-39, -1.8324e-39, -1.8324e-39],
          [-1.8324e-39,

epoch [9/100], loss:0.0000
x:  tensor([[[[ 4.3583e-23,  4.3583e-23,  4.3583e-23],
          [ 4.3583e-23,  4.3583e-23,  4.3583e-23],
          [ 4.3583e-23,  4.3583e-23,  4.3583e-23]],

         [[-2.7963e-13, -2.7963e-13, -2.7963e-13],
          [-2.7963e-13, -2.7963e-13, -2.7963e-13],
          [-2.7963e-13, -2.7963e-13, -2.7963e-13]]]],
       grad_fn=<MaxPool2DWithIndicesBackward>)
y: tensor([[[[ 3.4565e-23,  3.9043e-23,  3.9043e-23],
          [ 3.4565e-23,  3.9043e-23,  3.9043e-23],
          [ 3.0807e-23,  3.6976e-23,  3.7081e-23]],

         [[-2.7963e-13, -2.7963e-13, -2.7963e-13],
          [-2.7963e-13, -2.7963e-13, -2.7963e-13],
          [-2.7963e-13, -2.7963e-13, -2.7963e-13]]]],
       grad_fn=<MaxPool2DWithIndicesBackward>)
b: tensor([[[[ 8.6493e-23,  8.6743e-23,  8.6743e-23],
          [ 8.6493e-23,  8.6743e-23,  8.6743e-23],
          [ 8.6473e-23,  8.6743e-23,  8.6743e-23]],

         [[-2.7963e-13, -2.7963e-13, -2.7963e-13],
          [-2.7963e-13, -2.7963e-13, -2.7

epoch [17/100], loss:0.0000
x:  tensor([[[[-4.3417e-40, -3.1848e-17, -1.9450e-17],
          [-2.2074e-19, -2.9132e-16, -2.9126e-16],
          [-2.2277e-20, -2.8991e-16, -2.8947e-16]],

         [[ 1.9117e-06,  1.9117e-06,  1.9117e-06],
          [ 1.9117e-06,  1.9117e-06,  1.9117e-06],
          [ 1.9117e-06,  1.9117e-06,  1.9117e-06]]]],
       grad_fn=<MaxPool2DWithIndicesBackward>)
y: tensor([[[[-4.3417e-40, -5.0248e-18, -5.0248e-18],
          [-2.1520e-19, -2.9393e-16, -2.9618e-16],
          [-1.8183e-20, -2.9383e-16, -2.9418e-16]],

         [[ 1.9117e-06,  1.9117e-06,  1.9117e-06],
          [ 1.9117e-06,  1.9117e-06,  1.9117e-06],
          [ 1.9117e-06,  1.9117e-06,  1.9117e-06]]]],
       grad_fn=<MaxPool2DWithIndicesBackward>)
b: tensor([[[[-4.3417e-40, -6.0933e-18, -6.0933e-18],
          [-2.1109e-20, -3.7596e-17, -4.0192e-17],
          [-2.1109e-20, -3.7977e-17, -3.8543e-17]],

         [[ 1.9117e-06,  1.9117e-06,  1.9117e-06],
          [ 1.9117e-06,  1.9117e-06,  1.

epoch [25/100], loss:0.0000
x:  tensor([[[[ 3.6316e-17,  3.9254e-17,  3.9254e-17],
          [ 2.8571e-17,  2.8571e-17,  2.8571e-17],
          [ 2.8571e-17,  2.8571e-17,  2.8571e-17]],

         [[-1.5043e-10, -1.5043e-10, -1.5043e-10],
          [-1.5043e-10, -1.5043e-10, -1.5043e-10],
          [-1.5043e-10, -1.5043e-10, -1.5043e-10]]]],
       grad_fn=<MaxPool2DWithIndicesBackward>)
y: tensor([[[[ 3.5220e-17,  3.6328e-17,  3.6328e-17],
          [ 2.5061e-17,  2.5656e-17,  2.6834e-17],
          [ 2.5061e-17,  2.5656e-17,  2.6834e-17]],

         [[-1.5043e-10, -1.5043e-10, -1.5043e-10],
          [-1.5043e-10, -1.5043e-10, -1.5043e-10],
          [-1.5043e-10, -1.5043e-10, -1.5043e-10]]]],
       grad_fn=<MaxPool2DWithIndicesBackward>)
b: tensor([[[[ 3.7371e-17,  3.7801e-17,  3.9251e-17],
          [ 2.6560e-17,  2.6560e-17,  2.6560e-17],
          [ 2.6560e-17,  2.6560e-17,  2.6560e-17]],

         [[-1.5043e-10, -1.5043e-10, -1.5043e-10],
          [-1.5043e-10, -1.5043e-10, -1.

epoch [33/100], loss:0.0000
x:  tensor([[[[ 2.0639e-18,  2.0639e-18,  2.0639e-18],
          [ 2.0639e-18,  2.0639e-18,  2.0639e-18],
          [ 2.0639e-18,  2.0639e-18,  2.0639e-18]],

         [[-4.0084e-10, -4.0084e-10, -4.0084e-10],
          [-4.0084e-10, -4.0084e-10, -4.0084e-10],
          [-4.0084e-10, -4.0084e-10, -4.0084e-10]]]],
       grad_fn=<MaxPool2DWithIndicesBackward>)
y: tensor([[[[ 2.1245e-18,  2.1245e-18,  2.1244e-18],
          [ 2.1244e-18,  2.1244e-18,  2.1244e-18],
          [ 1.7556e-18,  1.7556e-18,  1.7556e-18]],

         [[-4.0084e-10, -4.0084e-10, -4.0084e-10],
          [-4.0084e-10, -4.0084e-10, -4.0084e-10],
          [-4.0084e-10, -4.0084e-10, -4.0084e-10]]]],
       grad_fn=<MaxPool2DWithIndicesBackward>)
b: tensor([[[[ 2.1782e-18,  2.1782e-18,  2.1782e-18],
          [ 2.1782e-18,  2.1782e-18,  2.1782e-18],
          [ 2.1782e-18,  2.1782e-18,  2.1782e-18]],

         [[-4.0084e-10, -4.0084e-10, -4.0084e-10],
          [-4.0084e-10, -4.0084e-10, -4.

...

epoch [65/100], loss:0.0000
x:  tensor([[[[-4.3417e-40, -9.8479e-20, -1.3152e-19],
          [ 2.2940e-21,  2.2940e-21, -2.9722e-20],
          [ 2.2940e-21,  2.2940e-21,  3.1369e-22]],

         [[ 1.2268e-04,  1.2268e-04,  1.2268e-04],
          [ 1.2268e-04,  1.2268e-04,  1.2268e-04],
          [ 1.2268e-04,  1.2268e-04,  1.2268e-04]]]],
       grad_fn=<MaxPool2DWithIndicesBackward>)
y: tensor([[[[-4.3417e-40, -1.0150e-19, -1.3062e-19],
          [ 2.2428e-21,  2.2428e-21, -2.9025e-20],
          [ 2.2428e-21,  2.2428e-21,  4.1374e-22]],

         [[ 1.2268e-04,  1.2268e-04,  1.2268e-04],
          [ 1.2268e-04,  1.2268e-04,  1.2268e-04],
          [ 1.2268e-04,  1.2268e-04,  1.2268e-04]]]],
       grad_fn=<MaxPool2DWithIndicesBackward>)
b: tensor([[[[-4.3417e-40, -9.8425e-20, -1.2988e-19],
          [ 2.2159e-21,  2.2159e-21, -2.8620e-20],
          [ 2.2159e-21,  2.2159e-21,  5.3831e-22]],

         [[ 1.2268e-04,  1.2268e-04,  1.2268e-04],
          [ 1.2268e-04,  1.2268e-04,  1.

epoch [73/100], loss:0.0000
x:  tensor([[[[1.8546e-19, 1.8546e-19, 1.8546e-19],
          [1.5478e-19, 1.5478e-19, 1.7697e-19],
          [1.3926e-19, 1.4008e-19, 1.7697e-19]],

         [[2.1762e-14, 2.1762e-14, 2.1762e-14],
          [2.1762e-14, 2.1762e-14, 2.1762e-14],
          [2.1762e-14, 2.1762e-14, 2.1762e-14]]]],
       grad_fn=<MaxPool2DWithIndicesBackward>)
y: tensor([[[[1.6491e-19, 1.6491e-19, 1.8015e-19],
          [1.6491e-19, 1.6491e-19, 1.8015e-19],
          [1.6491e-19, 1.6491e-19, 1.8015e-19]],

         [[2.1762e-14, 2.1762e-14, 2.1762e-14],
          [2.1762e-14, 2.1762e-14, 2.1762e-14],
          [2.1762e-14, 2.1762e-14, 2.1762e-14]]]],
       grad_fn=<MaxPool2DWithIndicesBackward>)
b: tensor([[[[1.8710e-19, 1.8865e-19, 1.9425e-19],
          [1.8710e-19, 1.8865e-19, 1.9425e-19],
          [1.8710e-19, 1.8865e-19, 1.8865e-19]],

         [[2.1762e-14, 2.1762e-14, 2.1762e-14],
          [2.1762e-14, 2.1762e-14, 2.1762e-14],
          [2.1762e-14, 2.1762e-14, 2.176

epoch [81/100], loss:0.0000
x:  tensor([[[[ 6.8829e-20,  4.6330e-20,  4.6326e-20],
          [ 6.8829e-20,  4.6330e-20,  4.6326e-20],
          [ 6.8829e-20,  4.6329e-20,  4.6329e-20]],

         [[-2.9455e-13, -2.9455e-13, -2.9455e-13],
          [-2.9455e-13, -2.9455e-13, -2.9455e-13],
          [-2.9455e-13, -2.9455e-13, -2.9455e-13]]]],
       grad_fn=<MaxPool2DWithIndicesBackward>)
y: tensor([[[[ 7.5906e-20,  5.4443e-20,  5.4440e-20],
          [ 7.5906e-20,  5.4443e-20,  5.4440e-20],
          [ 7.5906e-20,  5.4443e-20,  5.4440e-20]],

         [[-2.9455e-13, -2.9455e-13, -2.9455e-13],
          [-2.9455e-13, -2.9455e-13, -2.9455e-13],
          [-2.9455e-13, -2.9455e-13, -2.9455e-13]]]],
       grad_fn=<MaxPool2DWithIndicesBackward>)
b: tensor([[[[ 6.7643e-20,  6.7643e-20,  6.7642e-20],
          [ 6.7643e-20,  8.5637e-20,  8.5637e-20],
          [ 4.4206e-20,  8.6301e-20,  8.6301e-20]],

         [[-2.9455e-13, -2.9455e-13, -2.9455e-13],
          [-2.9455e-13, -2.9455e-13, -2.

epoch [89/100], loss:0.0000
x:  tensor([[[[-4.3417e-40, -6.9124e-20, -6.8138e-20],
          [-2.1354e-19, -2.1354e-19, -2.1354e-19],
          [-4.5578e-19, -5.2267e-19, -1.4221e-18]],

         [[ 5.7396e-06,  5.7396e-06,  5.7396e-06],
          [ 5.7396e-06,  5.7396e-06,  5.7396e-06],
          [ 5.7396e-06,  5.7396e-06,  5.7396e-06]]]],
       grad_fn=<MaxPool2DWithIndicesBackward>)
y: tensor([[[[-4.3417e-40, -7.4516e-20, -5.4853e-20],
          [-2.0033e-18, -3.6841e-19, -3.6841e-19],
          [-2.0039e-18, -1.9290e-18, -1.9290e-18]],

         [[ 5.7396e-06,  5.7396e-06,  5.7396e-06],
          [ 5.7396e-06,  5.7396e-06,  5.7396e-06],
          [ 5.7396e-06,  5.7396e-06,  5.7396e-06]]]],
       grad_fn=<MaxPool2DWithIndicesBackward>)
b: tensor([[[[-4.3417e-40, -3.7098e-20, -3.7098e-20],
          [-1.5037e-19, -2.0989e-19, -3.7632e-19],
          [-6.0083e-19, -6.6676e-19, -1.0679e-18]],

         [[ 5.7396e-06,  5.7396e-06,  5.7396e-06],
          [ 5.7396e-06,  5.7396e-06,  5.

epoch [97/100], loss:0.0000
x:  tensor([[[[ 4.2636e-18, -2.1877e-17, -1.4511e-17],
          [ 4.2636e-18, -2.1877e-17, -1.3448e-17],
          [ 4.2636e-18, -2.0091e-17, -1.3448e-17]],

         [[ 1.1127e-08,  1.1127e-08,  1.1127e-08],
          [ 1.1127e-08,  1.1127e-08,  1.1127e-08],
          [ 1.1127e-08,  1.1127e-08,  1.1127e-08]]]],
       grad_fn=<MaxPool2DWithIndicesBackward>)
y: tensor([[[[ 4.3699e-18, -2.0779e-17, -1.5389e-17],
          [ 4.3699e-18, -2.0779e-17, -1.5389e-17],
          [ 4.3699e-18, -2.0808e-17, -1.5336e-17]],

         [[ 1.1127e-08,  1.1127e-08,  1.1127e-08],
          [ 1.1127e-08,  1.1127e-08,  1.1127e-08],
          [ 1.1127e-08,  1.1127e-08,  1.1127e-08]]]],
       grad_fn=<MaxPool2DWithIndicesBackward>)
b: tensor([[[[ 4.1688e-18, -2.1023e-17, -1.3869e-17],
          [ 4.1688e-18, -2.1023e-17, -1.3866e-17],
          [ 4.1688e-18, -2.1207e-17, -1.3848e-17]],

         [[ 1.1127e-08,  1.1127e-08,  1.1127e-08],
          [ 1.1127e-08,  1.1127e-08,  1.